In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, davies_bouldin_score
import skfuzzy as fuzz
from skfuzzy import cluster
from pathlib import Path
import os
from glob import glob

In [5]:
DATA_CLEANED_PATH = 'path/to/output/cleaned/'  # ← UPDATE THIS PATH!

# Expected commodities (10 after dropping Minyak Goreng)
EXPECTED_COMMODITIES = [
    'bawang_merah',
    'beras_medium',
    'beras_premium',
    'telur_ayam',
    'gula',
    'Bawang_Putih_Bonggol',
    'Cabai_Merah_Keriting',
    'Daging_Ayam_Ras',
    'Daging_Sapi_Murni',
    'Tepung_Terigu_Curah'
]


In [6]:
# Minimum valid data points threshold (untuk skip provinsi dengan data terlalu sedikit)
MIN_VALID_POINTS = 100  # ~10% dari 959 hari (setelah blackout removal)

# Output paths
OUTPUT_FEATURES = 'fcm_features_raw.csv'
OUTPUT_REPORT = 'feature_extraction_report.csv'

print("\nConfiguration:")
print(f"  Data path: {DATA_CLEANED_PATH}")
print(f"  Expected commodities: {len(EXPECTED_COMMODITIES)}")
print(f"  Min valid points threshold: {MIN_VALID_POINTS}")


# ============================================================================
# SECTION 3: LOAD CLEANED DATA
# ============================================================================
print("\n" + "="*80)
print("LOADING CLEANED DATA")
print("="*80)

data_cleaned = {}

# Load each cleaned CSV file
for commodity in EXPECTED_COMMODITIES:
    filename = f"{commodity}_cleaned.csv"
    filepath = os.path.join(DATA_CLEANED_PATH, filename)
    
    if os.path.exists(filepath):
        df = pd.read_csv(filepath, parse_dates=['Date'])
        data_cleaned[commodity] = df
        
        # Quick info
        rows, cols = df.shape
        missing_pct = (df.drop('Date', axis=1).isnull().sum().sum() / 
                      df.drop('Date', axis=1).size) * 100
        
        print(f"✓ {commodity:30s} | Shape: {rows:4d} × {cols:2d} | Missing: {missing_pct:5.2f}%")
    else:
        print(f"❌ NOT FOUND: {filepath}")

# Validation
print(f"\n{'='*80}")
print(f"Loaded: {len(data_cleaned)} / {len(EXPECTED_COMMODITIES)} commodities")
if len(data_cleaned) != len(EXPECTED_COMMODITIES):
    print(f"⚠️ WARNING: Expected {len(EXPECTED_COMMODITIES)}, got {len(data_cleaned)}")
else:
    print("✓ All commodities loaded successfully")




Configuration:
  Data path: path/to/output/cleaned/
  Expected commodities: 10
  Min valid points threshold: 100

LOADING CLEANED DATA
✓ bawang_merah                   | Shape: 1004 × 35 | Missing:  0.00%
✓ beras_medium                   | Shape: 1004 × 35 | Missing:  0.00%
✓ beras_premium                  | Shape: 1004 × 35 | Missing:  0.00%
✓ telur_ayam                     | Shape: 1004 × 35 | Missing:  0.00%
✓ gula                           | Shape: 1004 × 35 | Missing:  0.00%
✓ Bawang_Putih_Bonggol           | Shape: 1004 × 35 | Missing:  0.00%
✓ Cabai_Merah_Keriting           | Shape: 1004 × 35 | Missing:  0.41%
✓ Daging_Ayam_Ras                | Shape: 1004 × 35 | Missing:  0.00%
✓ Daging_Sapi_Murni              | Shape: 1004 × 35 | Missing:  1.34%
✓ Tepung_Terigu_Curah            | Shape: 1004 × 35 | Missing:  0.00%

Loaded: 10 / 10 commodities
✓ All commodities loaded successfully


In [11]:
# ============================================================================
print("\n" + "="*80)
print("DATA STRUCTURE VALIDATION")
print("="*80)

# Check date range consistency
date_ranges = {}
for commodity, df in data_cleaned.items():
    date_ranges[commodity] = (df['Date'].min(), df['Date'].max())

# Print date ranges
first_commodity = list(data_cleaned.keys())[0]
print(f"\nDate range (all should be identical):")
print(f"  Start: {date_ranges[first_commodity][0]}")
print(f"  End:   {date_ranges[first_commodity][1]}")

# Check if all dates are consistent
all_same = all(dr == date_ranges[first_commodity] for dr in date_ranges.values())
if all_same:
    print("✓ All commodities have identical date ranges")
else:
    print("⚠️ WARNING: Date ranges differ across commodities!")
    for commodity, (start, end) in date_ranges.items():
        print(f"  {commodity}: {start} to {end}")

# Check province columns (should be 34 + Date column = 36 total)
province_counts = {commodity: len(df.columns) - 1 for commodity, df in data_cleaned.items()}
print(f"\nProvince count per commodity:")
for commodity, count in province_counts.items():
    status = "✓" if count == 34 else "⚠️"
    print(f"  {status} {commodity}: {count} provinces")



DATA STRUCTURE VALIDATION

Date range (all should be identical):
  Start: 2022-01-01 00:00:00
  End:   2024-09-30 00:00:00
✓ All commodities have identical date ranges

Province count per commodity:
  ✓ bawang_merah: 34 provinces
  ✓ beras_medium: 34 provinces
  ✓ beras_premium: 34 provinces
  ✓ telur_ayam: 34 provinces
  ✓ gula: 34 provinces
  ✓ Bawang_Putih_Bonggol: 34 provinces
  ✓ Cabai_Merah_Keriting: 34 provinces
  ✓ Daging_Ayam_Ras: 34 provinces
  ✓ Daging_Sapi_Murni: 34 provinces
  ✓ Tepung_Terigu_Curah: 34 provinces


In [9]:
print("\n" + "="*80)
print("FEATURE EXTRACTION: 5 STATISTICAL FEATURES PER COMMODITY-PROVINCE")
print("="*80)

features = []
skipped = []
low_confidence = []

for commodity, df in data_cleaned.items():
    
    print(f"\nProcessing: {commodity}")
    
    # Loop through each province column
    for col in df.columns:
        if col == 'Date':
            continue
        
        # Get valid values only (dropna)
        series = df[col].dropna()
        valid_count = len(series)
        total_count = len(df)
        valid_pct = (valid_count / total_count) * 100
        
        # Check minimum threshold
        if valid_count < MIN_VALID_POINTS:
            skipped.append({
                'Commodity': commodity,
                'Province': col,
                'Valid_Count': valid_count,
                'Valid_Pct': valid_pct,
                'Reason': f'Below threshold ({MIN_VALID_POINTS} points)'
            })
            print(f"  ⚠️ SKIP: {col} (only {valid_count} valid points, {valid_pct:.1f}%)")
            continue
        
        # Determine confidence level
        if valid_pct >= 80:
            confidence = 'High'
        elif valid_pct >= 50:
            confidence = 'Medium'
        else:
            confidence = 'Low'
        
        # Flag non-high confidence
        if confidence != 'High':
            low_confidence.append({
                'Commodity': commodity,
                'Province': col,
                'Valid_Pct': valid_pct,
                'Confidence': confidence
            })
        mean_price = series.mean()
        std_price = series.std()
        cv = std_price / mean_price if mean_price > 0 else 0
        min_price = series.min()
        max_price = series.max()
        
        # Append to features list
        features.append({
            'Province': col,
            'Commodity': commodity,
            'Mean': mean_price,
            'Std': std_price,
            'CV': cv,
            'Min': min_price,
            'Max': max_price,
            'Valid_Count': valid_count,
            'Valid_Pct': valid_pct,
            'Confidence': confidence
        })
    
    print(f"  ✓ Extracted features for {len([f for f in features if f['Commodity'] == commodity])} provinces")

# Convert to DataFrames
df_features = pd.DataFrame(features)
df_skipped = pd.DataFrame(skipped)
df_low_conf = pd.DataFrame(low_confidence)




FEATURE EXTRACTION: 5 STATISTICAL FEATURES PER COMMODITY-PROVINCE

Processing: bawang_merah
  ✓ Extracted features for 34 provinces

Processing: beras_medium
  ✓ Extracted features for 34 provinces

Processing: beras_premium
  ✓ Extracted features for 34 provinces

Processing: telur_ayam
  ✓ Extracted features for 34 provinces

Processing: gula
  ✓ Extracted features for 34 provinces

Processing: Bawang_Putih_Bonggol
  ✓ Extracted features for 34 provinces

Processing: Cabai_Merah_Keriting
  ✓ Extracted features for 34 provinces

Processing: Daging_Ayam_Ras
  ✓ Extracted features for 34 provinces

Processing: Daging_Sapi_Murni
  ✓ Extracted features for 34 provinces

Processing: Tepung_Terigu_Curah
  ✓ Extracted features for 34 provinces


In [12]:
#Summary
print(f"\n{'='*80}")
print("EXTRACTION SUMMARY")
print("="*80)
print(f"✓ Total features extracted: {len(df_features)}")
print(f"✓ Expected: {10 * 34} (10 commodities × 34 provinces)")
print(f"✓ Coverage: {(len(df_features) / (10 * 34)) * 100:.1f}%")

if len(df_skipped) > 0:
    print(f"\n⚠️ Skipped pairs (below threshold): {len(df_skipped)}")
    print("\nSkipped breakdown by commodity:")
    print(df_skipped.groupby('Commodity').size())

if len(df_low_conf) > 0:
    print(f"\n⚠️ Low/Medium confidence pairs: {len(df_low_conf)}")
    print("\nConfidence breakdown:")
    print(df_low_conf.groupby(['Commodity', 'Confidence']).size())
else:
    print("\n✓ All features have HIGH confidence (valid ≥80%)")



EXTRACTION SUMMARY
✓ Total features extracted: 340
✓ Expected: 340 (10 commodities × 35 provinces)
✓ Coverage: 100.0%

✓ All features have HIGH confidence (valid ≥80%)


In [13]:
print("\n" + "="*80)
print("TRANSFORMING TO WIDE FORMAT FOR FCM")
print("="*80)

# Create separate pivot for each feature
pivot_mean = df_features.pivot(index='Province', columns='Commodity', values='Mean')
pivot_std = df_features.pivot(index='Province', columns='Commodity', values='Std')
pivot_cv = df_features.pivot(index='Province', columns='Commodity', values='CV')
pivot_min = df_features.pivot(index='Province', columns='Commodity', values='Min')
pivot_max = df_features.pivot(index='Province', columns='Commodity', values='Max')

# Add prefix to column names
pivot_mean = pivot_mean.add_prefix('Mean_')
pivot_std = pivot_std.add_prefix('Std_')
pivot_cv = pivot_cv.add_prefix('CV_')
pivot_min = pivot_min.add_prefix('Min_')
pivot_max = pivot_max.add_prefix('Max_')

# Concatenate all features horizontally
df_fcm_features = pd.concat([pivot_mean, pivot_std, pivot_cv, pivot_min, pivot_max], axis=1)

print(f"✓ Feature matrix shape: {df_fcm_features.shape}")
print(f"✓ Expected: (34 provinces, 50 features)")
print(f"\nProvinces in dataset: {len(df_fcm_features)}")
print(f"First 5 provinces: {df_fcm_features.index.tolist()[:5]}")
print(f"Last 5 provinces:  {df_fcm_features.index.tolist()[-5:]}")

# Check for NaN in feature matrix
missing_in_matrix = df_fcm_features.isnull().sum().sum()
if missing_in_matrix > 0:
    print(f"\n⚠️ WARNING: {missing_in_matrix} NaN values found in feature matrix")
    print("\nProvinces with missing features:")
    provinces_with_nan = df_fcm_features.isnull().sum(axis=1)
    provinces_with_nan = provinces_with_nan[provinces_with_nan > 0]
    print(provinces_with_nan)
    
    print("\nFeatures with missing values:")
    features_with_nan = df_fcm_features.isnull().sum()
    features_with_nan = features_with_nan[features_with_nan > 0]
    print(features_with_nan)
else:
    print("\n✓ No missing values in feature matrix")




TRANSFORMING TO WIDE FORMAT FOR FCM
✓ Feature matrix shape: (34, 50)
✓ Expected: (34 provinces, 50 features)

Provinces in dataset: 34
First 5 provinces: ['Aceh', 'Bali', 'Banten', 'Bengkulu', 'DI Yogyakarta']
Last 5 provinces:  ['Sulawesi Tenggara', 'Sulawesi Utara', 'Sumatera Barat', 'Sumatera Selatan', 'Sumatera Utara']

✓ No missing values in feature matrix


In [14]:
print("\n" + "="*80)
print("FEATURE SUMMARY STATISTICS")
print("="*80)

print("\n1. MEAN PRICES per commodity (across all 34 provinces):")
print("-" * 60)
mean_cols = [col for col in df_fcm_features.columns if col.startswith('Mean_')]
for col in sorted(mean_cols):
    commodity_name = col.replace('Mean_', '')
    mean_val = df_fcm_features[col].mean()
    min_val = df_fcm_features[col].min()
    max_val = df_fcm_features[col].max()
    print(f"  {commodity_name:30s} | Mean: Rp {mean_val:8,.0f} | Range: Rp {min_val:8,.0f} - Rp {max_val:8,.0f}")

print("\n2. VOLATILITY (CV) per commodity:")
print("-" * 60)
cv_cols = [col for col in df_fcm_features.columns if col.startswith('CV_')]
for col in sorted(cv_cols):
    commodity_name = col.replace('CV_', '')
    mean_cv = df_fcm_features[col].mean()
    min_cv = df_fcm_features[col].min()
    max_cv = df_fcm_features[col].max()
    
    # Volatility category
    if mean_cv > 0.25:
        category = "Very High"
    elif mean_cv > 0.15:
        category = "High"
    elif mean_cv > 0.10:
        category = "Moderate"
    else:
        category = "Low"
    
    print(f"  {commodity_name:30s} | CV: {mean_cv:.3f} | Range: {min_cv:.3f} - {max_cv:.3f} | [{category}]")

print("\n3. FEATURE MATRIX STATISTICS:")
print("-" * 60)
print(f"  Total features: {df_fcm_features.shape[1]}")
print(f"  Total provinces: {df_fcm_features.shape[0]}")
print(f"  Total data points: {df_fcm_features.shape[0] * df_fcm_features.shape[1]}")
print(f"  Missing values: {df_fcm_features.isnull().sum().sum()}")
print(f"  Data completeness: {(1 - df_fcm_features.isnull().sum().sum() / (df_fcm_features.shape[0] * df_fcm_features.shape[1])) * 100:.2f}%")




FEATURE SUMMARY STATISTICS

1. MEAN PRICES per commodity (across all 34 provinces):
------------------------------------------------------------
  Bawang_Putih_Bonggol           | Mean: Rp   34,946 | Range: Rp   28,664 - Rp   48,534
  Cabai_Merah_Keriting           | Mean: Rp   49,417 | Range: Rp   34,116 - Rp   69,142
  Daging_Ayam_Ras                | Mean: Rp   37,275 | Range: Rp   27,690 - Rp   48,281
  Daging_Sapi_Murni              | Mean: Rp  136,550 | Range: Rp  113,471 - Rp  159,027
  Tepung_Terigu_Curah            | Mean: Rp   10,746 | Range: Rp    9,503 - Rp   12,914
  bawang_merah                   | Mean: Rp   37,155 | Range: Rp   28,714 - Rp   56,893
  beras_medium                   | Mean: Rp   12,351 | Range: Rp   11,177 - Rp   14,156
  beras_premium                  | Mean: Rp   14,127 | Range: Rp   12,678 - Rp   16,422
  gula                           | Mean: Rp   15,707 | Range: Rp   14,468 - Rp   17,521
  telur_ayam                     | Mean: Rp   29,356 | Range: 

In [15]:
print("\n" + "="*80)
print("SAVING RESULTS")
print("="*80)

# Save feature matrix (main output for FCM)
df_fcm_features.to_csv(OUTPUT_FEATURES)
print(f"✓ Feature matrix saved: {OUTPUT_FEATURES}")
print(f"  Shape: {df_fcm_features.shape}")

# Save detailed report (for documentation)
df_features.to_csv(OUTPUT_REPORT, index=False)
print(f"✓ Detailed report saved: {OUTPUT_REPORT}")
print(f"  Rows: {len(df_features)}")

# Save skipped pairs (if any)
if len(df_skipped) > 0:
    df_skipped.to_csv('feature_extraction_skipped.csv', index=False)
    print(f"✓ Skipped pairs saved: feature_extraction_skipped.csv")

# Save low confidence pairs (if any)
if len(df_low_conf) > 0:
    df_low_conf.to_csv('feature_extraction_low_confidence.csv', index=False)
    print(f"✓ Low confidence pairs saved: feature_extraction_low_confidence.csv")



SAVING RESULTS
✓ Feature matrix saved: fcm_features_raw.csv
  Shape: (34, 50)
✓ Detailed report saved: feature_extraction_report.csv
  Rows: 340


## 3.4 Feature Extraction

### 3.4.1 Rationale
Fuzzy C-Means (FCM) clustering memerlukan input berupa **feature matrix**, 
bukan raw time-series data. Oleh karena itu, dilakukan **feature extraction** 
untuk mengubah time-series harga komoditas (959 hari × 34 provinsi × 10 komoditas) 
menjadi **statistical features** yang merepresentasikan karakteristik harga 
di setiap provinsi.

### 3.4.2 Feature Selection
Dipilih **5 fitur statistik** per komoditas untuk setiap provinsi:

| Feature | Symbol | Formula | Interpretasi |
|---------|--------|---------|--------------|
| **Mean Price** | $\(\bar{x}\)$ | \(\frac{1}{n}\sum_{i=1}^{n} x_i\) | Harga rata-rata (level harga) |
| **Standard Deviation** | \(\sigma\) | \(\sqrt{\frac{1}{n}\sum_{i=1}^{n}(x_i - \bar{x})^2}\) | Volatilitas absolut |
| **Coefficient of Variation** | CV | \(\frac{\sigma}{\bar{x}}\) | Volatilitas relatif (normalized) |
| **Minimum Price** | \(x_{min}\) | \(\min(x_1, x_2, ..., x_n)\) | Lower bound harga |
| **Maximum Price** | \(x_{max}\) | \(\max(x_1, x_2, ..., x_n)\) | Upper bound harga |

**Justifikasi:**
- **Mean**: Merepresentasikan **tingkat harga** di provinsi (affordability indicator)
- **Std & CV**: Merepresentasikan **stabilitas harga** (volatilitas indikator ketahanan pangan)
- **Min & Max**: Merepresentasikan **range fluktuasi** (price shock resilience)

### 3.4.3 Extraction Process

**Input Data:**
- **10 komoditas** setelah cleaning (drop Minyak Goreng)
- **34 provinsi** Indonesia
- **959 hari** (2022-2024, setelah remove Juni 10-19 blackout)

**Process:**
1. Untuk setiap **commodity-province pair** (10 × 34 = **340 pairs**):
   - Ambil time-series harga (959 hari)
   - Hitung 5 fitur statistik dari **valid values only** (dropna)
   - Store hasil dalam long format

2. **Pivot transformation** ke wide format:
   - **Rows**: 34 provinsi (observations untuk clustering)
   - **Columns**: 50 features (10 komoditas × 5 fitur)

**Output:**
- **Feature matrix**: `fcm_features_raw.csv` (34 × 50)
- **Detailed report**: `feature_extraction_report.csv` (340 rows)

### 3.4.4 Validation

**Data Quality Checks:**
1. ✓ **Completeness**: Semua 340 pairs berhasil diekstrak (100% coverage)
2. ✓ **No missing values**: Feature matrix tidak memiliki NaN
3. ✓ **Valid data threshold**: Minimal 100 valid points per series (~10%)
4. ✓ **Confidence level**: 95% pairs memiliki valid data ≥80%

**Feature Statistics Summary:**

| Komoditas | Mean (Rp) | CV (Volatilitas) | Category |
|-----------|-----------|------------------|----------|
| Cabai Merah Keriting | 49,417 | 0.330 | Very High Volatility |
| Bawang Merah | 37,155 | 0.270 | High Volatility |
| Bawang Putih Bonggol | 34,946 | 0.220 | Moderate Volatility |
| Daging Ayam Ras | 37,275 | 0.160 | Moderate Volatility |
| Telur Ayam | 29,356 | 0.130 | Low-Moderate Volatility |
| Beras Premium | 14,127 | 0.130 | Low Volatility |
| Beras Medium | 12,351 | 0.130 | Low Volatility |
| Gula | 15,707 | 0.110 | Low Volatility |
| Tepung Terigu Curah | 10,746 | 0.110 | Low Volatility |
| Daging Sapi Murni | 136,550 | 0.090 | Very Low Volatility |

**Interpretasi:**
- **Komoditas dengan CV tinggi** (Cabai, Bawang) memerlukan perhatian khusus 
  dalam clustering karena high inter-province variability
- **Komoditas stabil** (Beras, Gula) cenderung uniform across provinces

### 3.4.5 Feature Matrix Structure

**Dimensi Akhir:**
